<img src="files/night_sky_eye_small.jpg">
# Image_comparison Script

Jessica Metzger (UChicago) and Jim Annis (Fermilab) 2018
Antonella Palmese and Jim Annis (Fermilab) 2019


To run this notebook:
  * you'll need to run 
       ** conda activate des18a"
  * really, just try sourcing setup.sh
       
 *(to run a notebook remotely, see 
 http://home.fnal.gov/~kadrlica/fnalstart.html
 
 ## How this should work
 
 * Given a search image:
 * Select from the 200mpc catalog the likely galaxies in the image as a list of ra,dec's
 * Select the galaxy from the bias subtracted search image, compare to BLISS single epoch images



# Do the Eyeballing!

This is the action cell!

For a given image, get the 
image name, ra, dec

Change the triplets,
then run the cell, it will take time to do all of them

In [29]:
import sauron
reload(sauron)

<module 'sauron' from 'sauron.pyc'>

In [31]:

# you do know the search image ra,dec, right? and filter? and name?
image_ra, image_dec =91.841025,-35.34666
image_name = "856770"; 
filter="r"
search_dir = "/data/des51.b/data/DTS/src/20190510/"

# get your 200mpc++ catalog into place
gal_cat = "/data/des60.b/data/palmese/GW_host_catalog/S190814bv/200Mpc_BPortal_2sigma_ranked.csv"
# tell me what columns to use
# ID dpDV RA DEC DIST Z ZERR CATTYPE
id_col = 0
ra_col = 2
dec_col = 3

# eyeball
sauron.work_image(search_dir, image_name, image_ra,image_dec, filter, gal_cat,
                  id_col_num=id_col, ra_col_num=ra_col, dec_col_num=dec_col, 
                  do_num=-1)

Opening file: /data/des51.b/data/DTS/src/20190510/DECam_00856770.fits.fz
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 
 Did 0 galaxies!

